In [1]:
!pip install gensim
!pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/67.1 kB ? eta -:--:--
     ---------------------------------------- 67.1/67.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20448 sha256=8ecf5232118be73cad5f61bc3594c8064e2b7f0dcc75584c38e9b605e228ef4f
  Stored in directory: c:\users\sonis\appdata\local\pip\cache\wheels\69\f5\e5\18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3522 sha256=eaf068b7151824e3170d720ea077da1d4b7aad42a0f6c42c3e19057792bdfe78
  Stored in directory: c:\users\sonis\appdata\local\pip\cache\wheels\9d\ff\2f\afe4cd56f47de147407705626517d68bea0f3b74eb1fb168e6
Succe

In [2]:
import gensim
import pandas as pd

### Reading and Exploring the Dataset
The dataset we are using here is a subset of Amazon reviews from the Cell Phones & Accessories category. The data is stored as a JSON file and can be read using pandas.


In [39]:
df = pd.read_json("Cell_Phones_and_Accessories_5.json", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [12]:
df.shape

(194439, 9)

### Simple Preprocessing & Tokenization
The first thing to do for any data science task is to clean the data.
For NLP, we apply various processing like converting all the words to lower case, trimming spaces, removing punctuations. 
This is something we will do over here too.

Additionally, we can also remove stop words like 'and', 'or', 'is', 'the', 'a', 'an' and convert words to their root forms like 'running' to 'run'.

In [14]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [15]:
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [16]:
review_text.loc[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [24]:
df.reviewText.loc[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

### Training the Word2Vec Model

Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

Workers define how many CPU threads to be used.

#### Initialize the model

In [25]:
#Gensim is a NLTK library which come with Word2VEc Class
model = gensim.models.Word2Vec(
    window=10,
    # At least 2 words must be present in a sentence in order to be consider for the training 
    min_count=2,
    # It define the number of CUP threads you want to use(core-processor 
    workers=4,
)


#### Build Vocabulary: Means building unique list of words



In [26]:
model.build_vocab(review_text, progress_per=1000)

In [33]:
#Epochs means how many time you iterate the whole training data set
model.epochs

5

#### Train the Word2Vec Model

In [28]:
model.corpus_count

194439

In [36]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61507596, 83868975)

### Save the Model

Save the model so that it can be reused in other applications

In [40]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

### Finding Similar Words and Similarity between words


In [41]:
model.wv.most_similar("bad")

[('shabby', 0.6735050082206726),
 ('terrible', 0.6699391007423401),
 ('good', 0.6064508557319641),
 ('horrible', 0.60100919008255),
 ('choosy', 0.5380186438560486),
 ('poor', 0.5360618829727173),
 ('legit', 0.5319231748580933),
 ('okay', 0.5318195819854736),
 ('disappointing', 0.5250775218009949),
 ('awful', 0.518735945224762)]

In [42]:
model.wv.similarity(w1="cheap", w2="inexpensive")

0.5342856

In [43]:
model.wv.similarity(w1="great", w2="good")

0.79246336

In [44]:
model.wv.similarity(w1="great", w2="great")

1.0